# State-Space Routines

Our package `StateSpaceRoutines.jl` implements some common routines for state-space models.
`DSGE.jl` relies on `StateSpaceRoutines.jl`'s filtering and smoothing functions, but the 
functionality provided is general enough to be used in other settings.

The provided algorithms are:

- Kalman filter (`kalman_filter`)
- Tempered particle filter (`tempered_particle_filter`): ["Tempered Particle Filtering"](https://federalreserve.gov/econresdata/feds/2016/files/2016072pap.pdf) (2016)
- Kalman smoothers:
  + `hamilton_smoother`: James Hamilton, [_Time Series Analysis_](https://www.amazon.com/Time-Analysis-James-Douglas-Hamilton/dp/0691042896) (1994)
  + `koopman_smoother`: S.J. Koopman, ["Disturbance Smoother for State Space Models"](https://www.jstor.org/stable/2336762) (_Biometrika_, 1993)
- Simulation smoothers:
  + `carter_kohn_smoother`: C.K. Carter and R. Kohn, ["On Gibbs Sampling for State Space Models"](https://www.jstor.org/stable/2337125) (_Biometrika_, 1994)
  + `durbin_koopman_smoother`: J. Durbin and S.J. Koopman, ["A Simple and Efficient Simulation Smoother for State Space Time Series Analysis"](https://www.jstor.org/stable/4140605) (_Biometrika_, 2002)

## Linear Estimation

### Linear State Space System

We assume a standard linear state space system, in which measumement error and fundamental shocks are 
independent. 

```
z_{t+1} = CCC + TTT*z_t + RRR*ϵ_t    (transition equation)
y_t     = DD  + ZZ*z_t  + η_t        (measurement equation)

ϵ_t ∼ N(0, QQ)
η_t ∼ N(0, EE)
Cov(ϵ_t, η_t) = 0
```


### Time-Invariant Methods

`StateSpaceRoutines.jl`'s linear filtering and smoothing functions handle both the single-regime and multiple-regimes cases. 


```
kalman_filter(data, TTT, RRR, CCC, QQ, ZZ, DD, EE, z0 = Vector(), P0 = Matrix(); allout = true, n_presample_periods = 0)

hamilton_smoother(data, TTT, RRR, CCC, QQ, ZZ, DD, EE, z0, P0; n_presample_periods = 0)
koopman_smoother(data, TTT, RRR, CCC, QQ, ZZ, DD, z0, P0, pred, vpred; n_presample_periods = 0)
carter_kohn_smoother(data, TTT, RRR, CCC, QQ, ZZ, DD, EE, z0, P0; n_presample_periods = 0, draw_states = true)
durbin_koopman_smoother(data, TTT, RRR, CCC, QQ, ZZ, DD, EE, z0, P0; n_presample_periods = 0, draw_states = true)
```

For more information, see the documentation for each function (e.g. by entering
`?kalman_filter` in the REPL).


First, let's consider a single-regime example using the `AnSchorfheide` model from `DSGE.jl`. This small, 3 equation New Keynesian model is described in "Bayesian Estimation of DSGE Models" by Sungbae An and Frank Schorfheide. 

In [1]:
using DSGE, StateSpaceRoutines
using QuantEcon: solve_discrete_lyapunov
using DataFrames, Plots

In [2]:
# Setup the model and data
m = AnSchorfheide()
df = readtable("us.txt", header = false, separator = ' ')
data = convert(Matrix{Float64}, df)'
dates = 1983:.25:2002.75 # data is from 1983Q1 to 2002Q4

params = [2.09, 0.98, 2.25, 0.65, 0.34, 3.16, 0.51, 0.81, 0.98, 0.93, 0.19, 0.65, 0.24,
          0.115985, 0.294166, 0.447587]
update!(m, params)

Dynamic Stochastic General Equilibrium Model
no. states:             8
no. anticipated shocks: 0
data vintage:           171219
description:
 Julia implementation of model defined in 'Bayesian Estimation of DSGE Models' by Sungbae An and Frank Schorfheide: AnSchorfheide, ss0


In [3]:
# Solution to a Linear DSGE Model w/ IID Gaussian Errors
system  = compute_system(m)
TTT     = system[:TTT]
RRR     = system[:RRR]
CCC     = system[:CCC]
EE      = system[:EE]
DD      = system[:DD]
ZZ      = system[:ZZ]
QQ      = system[:QQ]

3×3 Array{Float64,2}:
 0.0576  0.0     0.0   
 0.0     0.4225  0.0   
 0.0     0.0     0.0361

Running the `kalman_filter` will return the following items:
- `log_likelihood`: log likelihood of the state-space model
- `zend`: `Nz` x 1 final filtered state `z_{T|T}`
- `Pend`: `Nz` x `Nz` final filtered state covariance matrix `P_{T|T}`
- `pred`: `Nz` x `T` matrix of one-step predicted state vectors `z_{t|t-1}`
- `vpred`: `Nz` x `Nz` x `T` array of mean squared errors `P_{t|t-1}` of
  predicted state vectors
- `filt`: `Nz` x `T` matrix of filtered state vectors `z_{t|t}`
- `vfilt`: `Nz` x `Nz` x `T` matrix containing mean squared errors `P_{t|t}` of
  filtered state vectors
- `yprederror`: `Ny` x `T` matrix of observable prediction errors
  `y_t - y_{t|t-1}`
- `ystdprederror`: `Ny` x `T` matrix of standardized observable prediction errors
  `V_{t|t-1} \ (y_t - y_{t|t-1})`, where `y_t - y_{t|t-1} ∼ N(0, V_{t|t-1}`
- `rmse`: 1 x `T` row vector of root mean squared prediction errors
- `rmsd`: 1 x `T` row vector of root mean squared standardized prediction errors
- `z0`: `Nz` x 1 initial state vector. This may have reassigned to the last
  presample state vector if `n_presample_periods > 0`
- `P0`: `Nz` x `Nz` initial state covariance matrix. This may have reassigned to
  the last presample state covariance if `n_presample_periods > 0`
- `marginal_loglh`: a vector of the marginal log likelihoods from t = 1 to T

In [4]:
n_states = size(TTT, 1)
kal = kalman_filter(data, TTT, RRR, CCC, QQ, ZZ, DD, EE, zeros(n_states,), 1e6*eye(n_states, n_states))

# entry 6 is the n_periods by n_states matrix of filtered states
kal[6] 

8×80 Array{Float64,2}:
 -0.184703  -10.7754    -19.6144    …   3.06589     3.10173    3.09017 
 -0.72195     0.347567    0.193455      0.0519992  -0.227797  -0.311898
  0.778333    0.77998     0.913637     -0.90262    -0.901968  -0.974907
 -0.354339  -11.6059    -20.1197        2.70587     2.61902    2.97034 
  0.340308  -10.8586    -19.5903        2.90376     3.09887    3.12445 
  0.316583    0.646672    0.691767  …  -0.608649   -0.694255  -0.770598
  0.139365  -10.6002    -19.1972        2.89796     3.02754    3.03739 
 -0.207616    0.266262    0.217263     -0.106976   -0.230791  -0.278541

In [5]:
m.endogenous_states # display the list of states

DataStructures.OrderedDict{Symbol,Int64} with 8 entries:
  :y_t   => 1
  :π_t   => 2
  :R_t   => 3
  :y_t1  => 4
  :g_t   => 5
  :z_t   => 6
  :Ey_t1 => 7
  :Eπ_t1 => 8

Now that we have run the filter, we can see the filtered estimates for the time series of `z_t`, the technology process:

In [6]:
z_filt = kal[6][6,:]
plot(dates, z_filt, label = "filtered z")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1985 
 
 
 1990 
 
 
 1995 
 
 
 2000 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
<polyline clip-path="url(#clip4502)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.7332,140.945 52.5001,73.4361 59.2669,64.2134 66.0338,60.1235 72.8006,24.0262 79.5675,29.8612 86.3344,12.283 93.1012,63.9485 99.8681,90.948 106.635,112.151 
 113.402,135.41 120.169,107.402 126.936,145.458 133.702,244.357 140.469,198.901 147.236,190.827 154.003,155.945 160.77,145.127 167.537,142.722 174.304,146.604 
 181.07,166.534 187.837,129.643 194.604,99.8677 201.371,88.2857 208.138,57.2458 214.905,11.811 221.672,79.3474 228.438,84.6734 235.205,46.5734 241.972,96.1156 
 248.739,53.6514 255.506,64.6464 262.273,165.48 269.04,202.073 275.806,203.185 282.573,221.592 289.34,255.676 296.107,262.299 302.874,278.117 309.641,279.147 
 316.408,292.639 323.174,294.811 329.941,311.514 336.708,287.86 343.475,302.977 350.242,279.702 357.009,238.266 363.776,236.611 370.542,208.711 377.309,193.01 
 384.076,216.171 390.843,218.17 397.61,204.638 404.377,207.237 411.144,227.445 417.91,209.357 424.677,226.567 431.444,246.192 438.211,229.36 444.978,226.957 
 451.745,248.432 458.512,243.616 465.278,228.784 472.045,246.727 478.812,262.238 485.579,238.002 492.346,226.244 499.113,217.189 505.88,193.281 512.646,182.63 
 519.413,169.358 526.18,180.706 532.947,192.083 539.714,236.162 546.481,305.651 553.248,375.813 560.014,365.986 566.781,330.169 573.548,347.676 580.315,363.29 
 
 "/>
 
 
 
 
 filtered z

We can compare these filtered states with the smoothed results of the `hamilton_smoother`:

In [7]:
smoothed = hamilton_smoother(data, TTT, RRR, CCC, QQ, ZZ, DD, EE, zeros(n_states,), 1e6*eye(n_states, n_states))
smoothed_states = smoothed[1]
smoothed_shocks = smoothed[2]
z_smoothed = smoothed_states[6,:]

plot(dates, z_filt, label = "filtered z");
plot!(dates, z_smoothed, label = "smoothed z")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1985 
 
 
 1990 
 
 
 1995 
 
 
 2000 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
<polyline clip-path="url(#clip4702)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.7332,140.071 52.5001,73.0191 59.2669,63.8589 66.0338,59.7966 72.8006,23.9435 79.5675,29.7391 86.3344,12.2798 93.1012,63.5957 99.8681,90.4125 106.635,111.472 
 113.402,134.573 120.169,106.755 126.936,144.554 133.702,242.784 140.469,197.635 147.236,189.616 154.003,154.97 160.77,144.225 167.537,141.837 174.304,145.692 
 181.07,165.487 187.837,128.846 194.604,99.2718 201.371,87.7683 208.138,56.9384 214.905,11.811 221.672,78.8904 228.438,84.1804 235.205,46.3382 241.972,95.5452 
 248.739,53.3683 255.506,64.2889 262.273,164.44 269.04,200.786 275.806,201.89 282.573,220.173 289.34,254.026 296.107,260.604 302.874,276.315 309.641,277.338 
 316.408,290.739 323.174,292.896 329.941,309.486 336.708,285.992 343.475,301.007 350.242,277.889 357.009,236.734 363.776,235.09 370.542,207.379 377.309,191.784 
 384.076,214.788 390.843,216.773 397.61,203.333 404.377,205.915 411.144,225.986 417.91,208.02 424.677,225.114 431.444,244.606 438.211,227.888 444.978,225.501 
 451.745,246.83 458.512,242.047 465.278,227.316 472.045,245.138 478.812,260.543 485.579,236.471 492.346,224.793 499.113,215.799 505.88,192.053 512.646,181.474 
 519.413,168.292 526.18,179.563 532.947,190.863 539.714,234.644 546.481,303.663 553.248,373.35 560.014,363.589 566.781,328.014 573.548,345.404 580.315,360.911 
 
 "/>
<polyline clip-path="url(#clip4702)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.7332,132.276 52.5001,65.063 59.2669,58.09 66.0338,53.7194 72.8006,13.5458 79.5675,19.3237 86.3344,17.3001 93.1012,66.2126 99.8681,91.1228 106.635,107.953 
 113.402,129.944 120.169,102.701 126.936,139.245 133.702,238.984 140.469,195.114 147.236,190.073 154.003,154.585 160.77,144.029 167.537,141.399 174.304,145.667 
 181.07,162.056 187.837,123.913 194.604,94.1807 201.371,80.8565 208.138,55.3988 214.905,14.1039 221.672,81.0987 228.438,89.3635 235.205,50.4718 241.972,101.816 
 248.739,63.8182 255.506,74.9772 262.273,168.369 269.04,203.434 275.806,208.641 282.573,227.452 289.34,259.544 296.107,267.545 302.874,282.558 309.641,281.907 
 316.408,293.905 323.174,294.286 329.941,311.324 336.708,285.39 343.475,296.447 350.242,274.263 357.009,231.305 363.776,229.453 370.542,204.208 377.309,190.081 
 384.076,213.034 390.843,218.15 397.61,204.657 404.377,205.449 411.144,225.449 417.91,206.133 424.677,219.135 431.444,239.531 438.211,223.674 444.978,219.757 
 451.745,241.044 458.512,237.756 465.278,227.424 472.045,243.491 478.812,258.829 485.579,233.874 492.346,222.665 499.113,213.603 505.88,188.104 512.646,179.63 
 519.413,168.564 526.18,186.282 532.947,200.819 539.714,240.994 546.481,308.945 553.248,375.813 560.014,367.402 566.781,332.381 573.548,349.593 580.315,360.911 
 
 "/>
 
 
 
 
 filtered z 
 
 
 
 smoothed z

Here, the estimates of $z_{t|T}$ are approximately the same as $z_{t|t}$.  

### Regime-Switching Methods

All of the provided algorithms can handle time-varying state-space systems. To
do this, define `regime_indices`, a `Vector{Range{Int64}}` of length
`n_regimes`, where `regime_indices[i]` indicates the range of periods `t` in
regime `i`. Let `TTT_i`, `RRR_i`, etc. denote the state-space matrices in regime
`i`. Then the state space is given by:

```
z_{t+1} = CCC_i + TTT_i*z_t + RRR_i*ϵ_t    (transition equation)
y_t     = DD_i  + ZZ_i*z_t  + η_t          (measurement equation)

ϵ_t ∼ N(0, QQ_i)
η_t ∼ N(0, EE_i)
```

Letting `TTTs = [TTT_1, ..., TTT_{n_regimes}]`, etc., we can then call the time-
varying methods of the algorithms:

```
kalman_filter(regime_indices, data, TTTs, RRRs, CCCs, QQs, ZZs, DDs, EEs, z0 = Vector(), P0 = Matrix(); allout = true, n_presample_periods = 0)

hamilton_smoother(regime_indices, data, TTTs, RRRs, CCCs, QQs, ZZs, DDs, EEs, z0, P0; n_presample_periods = 0)
koopman_smoother(regime_indices, data, TTTs, RRRs, CCCs, QQs, ZZs, DDs, z0, P0, pred, vpred; n_presample_periods = 0)
carter_kohn_smoother(regime_indices, data, TTTs, RRRs, CCCs, QQs, ZZs, DDs, EEs, z0, P0; n_presample_periods = 0, draw_states = true)
durbin_koopman_smoother(regime_indices, data, TTTs, RRRs, CCCs, QQs, ZZs, DDs, EEs, z0, P0; n_presample_periods = 0, draw_states = true)
```

In DSGE.jl, we use this regime-switching framework to differentiate between the pre-forward guidance regime and the Fed's current regime.

As an example, we will filter our `AnSchorfheide` model on two regimes: 1983-1987 and 1988-2002. These correspond loosely to the pre-Great Moderation period and the Great Moderation. The regimes will be identical except that the variance of the monetary policy shock will be larger in the second:

In [9]:
m_2 = AnSchorfheide()
params_2 = [2.09, 0.98, 2.25, 0.65, 0.34, 3.16, 0.51, 0.81, 0.98, 0.93, 0.19*3, 0.65, 0.24,
          0.115985, 0.294166, 0.447587]
update!(m_2, params_2)

Dynamic Stochastic General Equilibrium Model
no. states:             8
no. anticipated shocks: 0
data vintage:           171219
description:
 Julia implementation of model defined in 'Bayesian Estimation of DSGE Models' by Sungbae An and Frank Schorfheide: AnSchorfheide, ss0


In [38]:
system_2  = compute_system(m)
TTT_2     = system_2[:TTT]
RRR_2     = system_2[:RRR]
CCC_2     = system_2[:CCC]
EE_2      = system_2[:EE]
DD_2      = system_2[:DD]
ZZ_2      = system_2[:ZZ]
QQ_2      = system_2[:QQ]

TTTs = [TTT_2, TTT]
RRRs = [RRR_2, RRR]
CCCs = [CCC_2, CCC]
EEs  = [EE_2, EE]
DDs  = [DD_2, DD]
ZZs  = [ZZ_2, ZZ]
QQs  = [QQ_2, QQ]

regime_indices = Range{Int64}[1:16,17:80]

2-element Array{Range{Int64},1}:
 1:16 
 17:80

In [ ]:
kal_regime = kalman_filter(regime_indices, data, TTTs, RRRs, CCCs, QQs, ZZs, DDs, EEs, 
                           zeros(n_states,), 1e6*eye(n_states, n_states))
smoothed_regime = hamilton_smoother(regime_indices, data, TTTs, RRRs, CCCs, QQs, ZZs, DDs, EEs, 
                                    zeros(n_states,), 1e6*eye(n_states, n_states))

In [ ]:
z_filt_regime = kal_regime[6][6,:]
z_smoothed_regime = smoothed_regime[1][6,:]

plot(dates, z_filt_regime, label = "filtered z");
plot!(dates, z_smoothed_regime, label = "smoothed z")

## Nonlinear Estimation

The tempered particle filter is a particle filtering method which can approximate the log-likelihood value implied by a general (potentially non-linear) state space system with the following representation:

### General State Space System
```
z_{t+1} = Φ(z_t, ϵ_t)        (transition equation)
y_t     = Ψ(z_t) + u_t       (measurement equation)

ϵ_t ∼ F_ϵ(∙; θ)
u_t ∼ N(0, HH), where HH is the variance matrix of the i.i.d measurement error
Cov(ϵ_t, u_t) = 0
```
- The documentation and code are located in [src/filters/tempered_particle_filter](https://github.com/FRBNY-DSGE/StateSpaceRoutines.jl/tree/doc/src/filters/tempered_particle_filter).
- The example is located in [docs/examples/tempered_particle_filter](https://github.com/FRBNY-DSGE/StateSpaceRoutines.jl/tree/doc/docs/examples/tempered_particle_filter)

In [42]:
# Define the (potentially non-linear) transition and measurement equations
Φ(s_t::Vector{Float64}, ϵ_t::Vector{Float64}) = TTT*s_t + RRR*ϵ_t + CCC
Ψ(s_t::Vector{Float64}, u_t::Vector{Float64}) = ZZ*s_t + DD + u_t

F_ϵ = Distributions.MvNormal(zeros(size(QQ, 1)), QQ)
F_u = Distributions.MvNormal(zeros(size(EE, 1)), EE)

FullNormal(
dim: 3
μ: [0.0, 0.0, 0.0]
Σ: [0.0134524 0.0 0.0; 0.0 0.0865339 0.0; 0.0 0.0 0.200334]
)
